In [20]:
    dir = 'CILroadseg'
    try:
        from google.colab import drive

        print('Colab environment detected. Mounting drive...')
        drive.mount('/content/drive')

        print('Mounted. Switching to directory... ', end = '')
        os.chdir('/content/drive/''My Drive''/'+ dir)
        print('done.')
    except:
        print('Colab environment not found. Working on ordinary directory.')

Colab environment detected. Mounting drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted. Switching to directory... done.


In [0]:
import numpy as np
import sys
import os
import matplotlib.image as mpimg

from util.submit import *
from util.helpers import *
from util.visualize import *
from util.cross_validation import cross_validate

## Loading Training Data

In [30]:
image_dir = "training/images/"
gt_dir = "training/groundtruth/"
test_dir = "test/images/"



X = []
Y = []
X_test = []


print("Loading training input...")
files = os.listdir(image_dir)
numfiles = len(files)
k = 0
for file in files:
  X.append(np.asarray(load_image((image_dir + file))))
  k += 1

  sys.stdout.write("\rProgress: "+ str(k * 100 // numfiles) +"%")

print("\rProgress: done (" + str(len(X)) + " images).")
X = np.array(X)

print("Loading training groundtruth...")
files = os.listdir(gt_dir)
numfiles = len(files)
k = 0
for file in files:
  Y.append(np.asarray(load_image((gt_dir + file))))
  k += 1

  sys.stdout.write("\rProgress: "+ str(k * 100 // numfiles) +"%")

print("\rProgress: done (" + str(len(Y)) + " images).")
Y = (np.array(Y) >= 0.25) * 1 # compensates for lossy image data


print("Loading test input...")
files = os.listdir(test_dir)
numfiles = len(files)
k = 0
for file in files:
  X_test.append(np.asarray(load_image((test_dir + file))))
  k += 1

  sys.stdout.write("\rProgress: "+ str(k * 100 // numfiles) +"%")

print("\rProgress: done (" + str(len(X_test)) + " images).")
X_test = np.array(X_test)


print()
print("       Training data shape: " + str(X.shape))
print("Training groundtruth shape: " + str(Y.shape))
print("           Test data shape: " + str(X_test.shape))

Loading training input...
Progress: done (100 images).
Loading training groundtruth...
Progress: done (100 images).
Loading test input...
Progress: done (94 images).

       Training data shape: (100, 400, 400, 3)
Training groundtruth shape: (100, 400, 400)
           Test data shape: (94, 608, 608, 3)


## Decomposer + ResNet/CNN

In [0]:
import tensorflow as tf
from tensorflow import keras

from decomposer import *
from resnet import *
from cnn import *

In [0]:
model = Decomposer(CNNModel())

In [0]:
choice = 'cv'

np.random.seed(1) #fix randomness
if choice = 'cv':
  cross_validate(model, 4, X, Y)

elif choice = 'tr':
  model.initialize()
  model.train(Y, X) 

else:
  model.load("saves/final/cnn.h5")

## Model Saving

In [0]:
weights_path = "saves/final/cnn.h5"

print("[Target file: "+ weights_path +"]")
print("Saving model to disk...", end = '')
model.save(weights_path)
print("done.")

In [0]:
test_masks_path = "test/pred/"
test_dir = "test/images/"

print("Predicting test cases... ", end = '')
for file in os.listdir(test_dir):
  img_id = int(re.search(r"\d+", file).group(0))

  X_test = np.array([np.asarray(load_image((test_dir + file)))])
  Y_pred = model.classify(X_test)

  save_image(np.repeat(Y_pred[0][:,:,np.newaxis], 3, axis=2), test_masks_path + "mask_"+ str(img_id) +".png")

print("done.")


In [0]:
image_paths = [test_masks_path + file for file in os.listdir(test_masks_path)]

masks_to_submission("test/subs/cnn.csv", image_paths)

In [0]:
test_dir = "test/images/"

print("Loading test input... ", end = '')
X_test = [np.asarray(load_image((test_dir + file))) for file in os.listdir(test_dir)]
print("done (" + str(len(X_test)) + " images).")
X_test = np.array(X_test)

In [0]:
Y_pred = model.classify(X_test[0:10])

view_image_mask(X_test[4], Y_pred[4])